In [30]:
import pandas as pd

In [31]:
df = pd.read_csv('dat.csv')

In [32]:
df

,Bharatiya Janata Party: (Delhi),Indian National Congress: (Delhi),Aam Aadmi Party: (Delhi),Bharatiya Janata Party: (Goa),Indian National Congress: (Goa),Bharatiya Janata Party: (Uttarakhand),Indian National Congress: (Uttarakhand),Bharatiya Janata Party: (Haryana),Indian National Congress: (Haryana),Bharatiya Janata Party: (Uttar Pradesh),Indian National Congress: (Uttar Pradesh),Samajwadi Party: (Uttar Pradesh),Bahujan Samaj Party: (Uttar Pradesh),Bharatiya Janata Party: (Madhya Pradesh),Indian National Congress: (Madhya Pradesh)
0,33.0000,14.0000,7.0000,33.0000,22.0000,19.0000,15.0000,14.0000,14.0000,47.0000,15.0000,13.0000,9.0000,44.0000,37.0000
1,35.0000,51.0000,10.0000,21.0000,32.0000,33.0000,42.0000,32.0000,32.0000,56.0000,42.0000,14.0000,10.0000,47.0000,62.0000
2,46.0000,46.0000,8.0000,31.0000,21.0000,52.0000,38.0000,24.0000,26.0000,70.0000,38.0000,13.0000,9.0000,37.0000,62.0000
3,40.0000,35.0000,9.0000,21.0000,0.0000,45.0000,32.0000,36.0000,15.0000,64.0000,31.0000,15.0000,14.0000,46.0000,44.0000
4,28.0000,31.0000,18.0000,20.0000,0.0000,46.0000,37.0000,31.0000,20.0000,43.0000,18.0000,11.0000,10.0000,39.0000,42.0000
5,53.0000,31.0000,21.0000,44.0000,22.0000,50.0000,18.0000,55.0000,19.0000,58.0000,21.0000,15.0000,12.0000,53.0000,33.0000
6,41.0000,33.0000,9.0000,27.0000,28.0000,34.0000,24.0000,49.0000,31.0000,54.0000,25.0000,13.0000,8.0000,37.0000,40.0000
7,90.0000,25.0000,11.0000,98.0000,0.0000,93.0000,37.0000,67.0000,33.0000,100.0000,29.0000,15.0000,14.0000,82.0000,40.0000
8,53.0000,22.0000,7.0000,0.0000,41.0000,78.0000,18.0000,41.0000,17.0000,66.0000,19.0000,11.0000,13.0000,60.0000,29.0000
9,53.0000,33.0000,12.0000,41.0000,62.0000,56.0000,47.0000,41.0000,29.0000,70.0000,31.0000,15.0000,12.0000,51.0000,34.0000


In [33]:
dataset = df.values

In [34]:
dataset = dataset.transpose()
dataset.shape

(15, 53)

In [35]:
X = dataset[:,:51]

In [36]:
Y = dataset[:,52]

In [37]:
X.dtype

dtype('float64')

In [38]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [39]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(51, 100) 
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 20)
        self.fc4 = nn.Linear(20, 1)

    def forward(self, x):
        # The hidden layers use ReLu activation function while the final layer uses sigmoid 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [57]:
totalset = []
# X = torch.from_numpy(X)
# print(X.dtype)
# X = X.to(torch.float32)
# print(X.dtype)
# Y = torch.from_numpy(Y)
# Y = Y.to(torch.float32)
for i in range(len(X)):
    totalset.append((X[i]/100,[[Y[i]]]))
totalset = np.array(totalset)
np.random.shuffle(totalset)
super_train_X = torch.Tensor([k[0] for k in totalset]).view(-1,1,51)
super_train_y = torch.Tensor([k[1] for k in totalset])

In [58]:
val_size = len(super_train_X)//3
train_X = super_train_X[:-val_size]
train_y = super_train_y[:-val_size]
test_X = super_train_X[-val_size:]
test_y = super_train_y[-val_size:]

In [59]:
net = Net()
# Define the loss function to be Binary Cross Entropy
loss_function = nn.BCELoss()
# Use RMS propogation as the optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [60]:
for epoch in (range(20)): # 3 full passes over the data
    lss = []
    for i in (range(len(train_X))):
        # Set the gradients to zero
        net.zero_grad()
        # Calculate output for the current record
        output = net(train_X[i].view(-1,51))
        # Calculate loss
        loss = loss_function(output, train_y[i])
        # Backpropogate
        loss.backward()
        lss.append(loss)
        # Adjust the hyper-parameters
        optimizer.step()
    avg = 0
    for item in lss:
        avg+=item.item()/len(lss)
    print("Average loss: "+ str(avg))

Average loss: 0.6722026586532592
Average loss: 0.6708502411842345
Average loss: 0.6695825099945069
Average loss: 0.6683930158615112
Average loss: 0.667276781797409
Average loss: 0.666228973865509
Average loss: 0.665245097875595
Average loss: 0.6643210113048553
Average loss: 0.6634528875350952
Average loss: 0.6626371085643769
Average loss: 0.6618704617023469
Average loss: 0.661149787902832
Average loss: 0.6604722142219545
Average loss: 0.6598350763320923
Average loss: 0.6592361569404601
Average loss: 0.6586726665496826
Average loss: 0.6581423103809357
Average loss: 0.6576428890228272
Average loss: 0.6571727454662323
Average loss: 0.656730091571808


In [61]:
correct = 0
total = 0

# Temproarily set gradients off as we are not training the model
with torch.no_grad():
    lss = []
    # Iterate through validation set
    for i in range(len(test_X)):
        # Calculate output for current record
        output = net(test_X[i].view(-1,51))
        # Float value of the output
        # Check if correct, update the variables accordingly
        print((output,test_y[i]))
#         loss = loss_function(output, train_y[i])
        loss = abs(output[0][0]-train_y[i][0][0])
        lss.append(loss)
#         if (output==test_y[i]):
#             correct += 1    
#         total += 1
# Calculate metrics
# acc = round(correct/total, 3)*100
    avg = 0
    for item in lss:
        avg+=item.item()/len(lss)
    print("Average loss: "+ str(avg))

(tensor([[0.4110]]), tensor([[0.1753]]))
(tensor([[0.4078]]), tensor([[0.5204]]))
(tensor([[0.4113]]), tensor([[0.1959]]))
(tensor([[0.4078]]), tensor([[0.5979]]))
(tensor([[0.4083]]), tensor([[0.6162]]))
Average loss: 0.17744367718696594
